In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import (
    clsf_boz_cd, clsf_pdgr_cd,
)

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/보험미수금')
BASE_YYMM = '202012'

In [3]:
# 데이터 불러오기
## 재보험미결명세
재보험미결명세 = pd.read_excel(FILE_PATH / '재보험미결명세_202012.xlsx',
    dtype={'보험종목': str, '계상일자': str, '재보험자': str, 'Broker': str, '회계지점': str, '처리지점': str})
# 재보험미결명세.to_csv(FILE_PATH / '재보험미결명세.csv', index=False)
# 재보험미결명세 = pd.read_csv(FILE_PATH / '재보험미결명세.csv',
#     dtype={'보험종목': str, '계상일자': str, '재보험자': str, 'Broker': str, '회계지점': str, '처리지점': str})

## 기초정보, 가정 등
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})

In [88]:
# 일반
재보험미결명세_일반_가공 = 재보험미결명세.loc[lambda x: x['보험종목'].str.startswith('1')].reset_index(drop=True)
재보험미결명세_일반_가공['DMFR_DVCD'] = '01'
재보험미결명세_일반_가공.loc[lambda x: (x['업무파트'] == '미주지원') | x['보험종목'].isin(['10900', '10901']), 'DMFR_DVCD'] = '02'
재보험미결명세_일반_가공 = 재보험미결명세_일반_가공.assign(ARC_INPL_CD = lambda x: x['보험종목']+ '10')
재보험미결명세_일반_가공['BOZ_CD'] = clsf_boz_cd(재보험미결명세_일반_가공, 일반_상품정보)
재보험미결명세_일반_가공['PDGR_CD'] = clsf_pdgr_cd(재보험미결명세_일반_가공, 일반_상품정보)
재보험미결명세_일반_가공 = 재보험미결명세_일반_가공 \
    .assign(
        부채구분 = lambda x: np.where(x['처리과목명'].str.contains('보험금'), '발생사고', '잔여보장'),
        미수미지급구분 = lambda x: np.where(x['계정과목명'].str.contains('(미수금|예탁금)', regex=True), '미수금', np.where(x['계정과목명'].str.contains('(미지급금|예수금)', regex=True), '미지급금', '#')),
        RRNR_DVCD = lambda x: np.where(x['처리과목명'].str.contains('수재', regex=True), '02', '03')
    )

일반_재보험미수미지급금 = 재보험미결명세_일반_가공.copy()
일반_재보험미수미지급금.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
일반_재보험미수미지급금.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
일반_재보험미수미지급금 = 일반_재보험미수미지급금.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', '부채구분', '미수미지급구분'])['원화잔액'].apply(lambda x: np.sum(x)/1000).reset_index()
일반_재보험미수미지급금.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,RRNR_DVCD,DMFR_DVCD,BOZ_CD,부채구분,미수미지급구분,원화잔액
0,03,01,A001,발생사고,미수금,"1,329,809"
1,03,01,A001,잔여보장,미지급금,"5,128,127"
2,03,01,A002,발생사고,미수금,"2,461,912"
3,03,01,A002,잔여보장,미지급금,"3,721,905"
4,03,01,A003,발생사고,미수금,"21,861,243"


In [87]:
# 자동차
재보험미결명세_자동차_가공 = 재보험미결명세.loc[lambda x: x['보험종목'].str.startswith('2')].reset_index(drop=True)
재보험미결명세_자동차_가공['DMFR_DVCD'] = '01'
재보험미결명세_자동차_가공
재보험미결명세_자동차_가공 = 재보험미결명세_자동차_가공 \
    .assign(
        부채구분 = lambda x: np.where(x['처리과목명'].str.contains('보험금'), '발생사고', '잔여보장'),
        미수미지급구분 = lambda x: np.where(x['계정과목명'].str.contains('(미수금|예탁금)', regex=True), '미수금', np.where(x['계정과목명'].str.contains('(미지급금|예수금)', regex=True), '미지급금', '#')),
        RRNR_DVCD = lambda x: np.where(x['처리과목명'].str.contains('수재', regex=True), '02', '03'),
drop=True)
재보험미결명세_자동차_가공['DMFR_DVCD'] = '01'
재보험미결명세_자동차_가공
재보험미결명세_자동차_가공 = 재보험미결명세_자동차_가공 \
    .assign(
        부채구분 = lambda x: np.where(x['처리과목명'].str.contains('보험금'), '발생사고', '잔여보장'),
        미수미지급구분 = lambda x: np.where(x['계정과목명'].str.contains('(미수금|예탁금)', regex=True), '미수금', np.where(x['계정과목명'].str.contains('(미지급금|예수금)', regex=True), '미지급금', '#')),
        RRNR_DVCD = lambda x: np.where(x['처리과목명'].str.contains('수재', regex=True), '02', '03'),
        BOZ_CD = lambda x: np.where(x['보험종목']=='20001', 'B002', np.where(x['보험종목']=='20045', 'B001', '#')),
    )

자동차_재보험미수미지급금 = 재보험미결명세_자동차_가공.copy()
자동차_재보험미수미지급금.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
자동차_재보험미수미지급금.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
자동차_재보험미수미지급금 = 자동차_재보험미수미지급금.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', '부채구분', '미수미지급구분'])['원화잔액'].apply(lambda x: np.sum(x)/1000).reset_index()
자동차_재보험미수미지급금.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,RRNR_DVCD,DMFR_DVCD,BOZ_CD,부채구분,미수미지급구분,원화잔액
0,03,01,B001,잔여보장,미지급금,50
1,03,01,B002,발생사고,미수금,"3,021,332"
2,03,01,B002,잔여보장,미지급금,"2,884,015"


In [92]:
# 데이터 내보내기
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/재보험미수미지급금_{now}.xlsx', 'xlsxwriter') as writer:
    일반_재보험미수미지급금.to_excel(writer, '일반_재보험미수미지급금', index=False)
    자동차_재보험미수미지급금.to_excel(writer, '자동차_재보험미수미지급금', index=False)